# Thu thập dữ liệu cổ phiếu Amazon từ trang web
https://finance.yahoo.com/quote/AMZN/history?period1=1495324800&period2=1653091200&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true

In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import pandas as pd
from selenium.common import NoSuchElementException
import time

    Tạo ra một webdriver để truy cập vào web với các options: web ẩn danh và window size là 1920x1080

In [2]:
chrome_options = Options()
chrome_options.add_argument("--incognito")
chrome_options.add_argument("--window-size=1920x1080")

In [3]:

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)


    Đưa link web vào driver để lấy được dữ liệu

In [4]:
urls = {
    "Amazon": "https://finance.yahoo.com/quote/AMZN/history/?period1=1449792000&period2=1731370523",
    "Tesla": "https://finance.yahoo.com/quote/TSLA/history/?period1=1449792000&period2=1731369600",
    "Microsoft": "https://finance.yahoo.com/quote/MSFT/history/?period1=1449792000&period2=1731369600"
}


In [5]:



def scrape_data(url):
    driver.get(url)
    time.sleep(5)  # Allow time for the page to load
    
    try:
        table_container = driver.find_element("css selector", "div.yf-j5d1ld.table-container")
        print("Table container found.")
    except NoSuchElementException:
        print("Table container not found.")
        return None
    
    Rows = table_container.find_elements("tag name", "tr")
    
    data = []
    for row in Rows:
        cols = row.find_elements("tag name", "td")
        if len(cols) == 7:  # Ensure there are exactly 7 columns
            RowDict = {
                "Date": cols[0].text.replace(',', ''),
                "Open": cols[1].text.replace(',', ''),
                "High": cols[2].text.replace(',', ''),
                "Low": cols[3].text.replace(',', ''),
                "Close": cols[4].text.replace(',', ''),
                "Adj Close": cols[5].text.replace(',', ''),
                "Volume": cols[6].text.replace(',', '')
            }
            data.append(RowDict)
    
    return pd.DataFrame(data)

In [6]:
for company, url in urls.items():
    df = scrape_data(url)
    if df is not None and not df.empty:
        df.to_csv(f'rawdata_{company}.csv', index=False)
        print(f"Data for {company} saved to rawdata_{company}.csv")
    else:
        print(f"No data found for {company}.")

# Close the driver after scraping
driver.quit()

Table container found.
Data for Amazon saved to rawdata_Amazon.csv
Table container found.
Data for Tesla saved to rawdata_Tesla.csv
Table container found.
Data for Microsoft saved to rawdata_Microsoft.csv


    Dùng BeautifulSoup để tiến hành phân tích cú pháp web

    Lấy ra thành phần table với class là 'W(100%) M(0)' lưu vào trong biến Table

    Lấy ra các hàng có thẻ tr với class là 'BdT Bdc($seperatorColor) Ta(end) Fz(s) Whs(nw)' lưu vào trong Rows

    Lấy dữ liệu của các cột trong từng thẻ tr

    Chuyển dictionary extracted_data thành một Dataframe và lưu lại thành file raw_data.csv